In [ ]:
!pip install -U sagemaker

In [2]:
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.huggingface import HuggingFace

In [13]:
#from sagemaker.huggingface import get_huggingface_llm_image_uri

In [4]:
role = sagemaker.get_execution_role()

In [5]:
role

'arn:aws:iam::385046010545:role/SageMakerLLMRole'

In [6]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [7]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./scripts",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-santosh/models/",
    hyperparameters=hyperparameters
)

In [8]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-santosh/datasets/"
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-16-09-59-43-092


2025-11-16 09:59:43 Starting - Starting the training job
2025-11-16 09:59:43 Pending - Training job waiting for capacity...
2025-11-16 10:00:05 Pending - Preparing the instances for training...
2025-11-16 10:00:33 Downloading - Downloading input data...
2025-11-16 10:00:54 Downloading - Downloading the training image........................
2025-11-16 10:04:51 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be

In [ ]:
# estimator.latest_training_job.model_data

In [21]:
estimator.model_data

's3://llm-model-artifacts-santosh/models/huggingface-pytorch-training-2025-11-16-09-59-43-092/output/model.tar.gz'

In [ ]:
# Use this code to check all the accessible services inside your AWS Sagemaker

# from sagemaker import image_uris

# image_uris.retrieve(
#     framework="huggingface",
#     region="ap-south-1",   # change your region
#     version="4.37.0",
#     image_scope="inference"
# )

In [ ]:
 # instance_type="ml.g5.xlarge",

In [ ]:
# model = HuggingFaceModel(
#     model_data="s3://bucket/model.tar.gz",
#     role=role,
#     entry_point="inference.py",
#     source_dir="inference",
#     transformers_version="4.36",
#     pytorch_version="2.1",
#     py_version="py310"
# )

In [ ]:
# ## the code for the deployment
# import sagemaker
# from sagemaker.huggingface import HuggingFaceModel

# role = sagemaker.get_execution_role()

# model = HuggingFaceModel(
#     model_data="s3://llm-model-artifacts-sunny/models/huggingface-pytorch-training-2025-11-15-11-42-14-536/output/model.tar.gz",
#     role=role,
#     transformers_version="4.37.0",
#     pytorch_version="2.1.0",
#     py_version="py310",
# )

# predictor = model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.m5.xlarge",
#     endpoint_name="live-finetune-endpoint"
# )

In [ ]:
from sagemaker.huggingface import HuggingFaceModel

model = HuggingFaceModel(
    model_data="s3://llm-model-artifacts-santosh/models/huggingface-pytorch-training-2025-11-16-09-59-43-092/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    source_dir="./inference",
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310"
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    endpoint_name="santosh2-finetune-endpoint"
)


INFO:sagemaker:Repacking model artifact (s3://llm-model-artifacts-santosh/models/huggingface-pytorch-training-2025-11-16-09-59-43-092/output/model.tar.gz), script artifact (./scripts), and dependencies ([]) into single tar.gz file located at s3://sagemaker-ap-south-1-385046010545/huggingface-pytorch-inference-2025-11-16-12-41-16-475/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2025-11-16-12-41-18-163
INFO:sagemaker:Creating endpoint-config with name santosh2-finetune-endpoint
INFO:sagemaker:Creating endpoint with name santosh2-finetune-endpoint


--------

In [28]:
## THIS IS JUST TO VALIDATE WHETHER MODEL WORKING OR NOT IN THE NOTEBOOK ITSELF
predictor.predict({"inputs": "Explain AWS S3"})

{'generated_text': 'Explain AWS S3 Bucket?\nAWS S3 is a cloud storage service that allows users to store, manage, and distribute large amounts of data. S3 buckets are logical units of storage that are managed by AWS and can be shared by multiple users.\nS3 buckets are designed to be scalable, reliable, and fast. They can be used to store static or dynamic content, such as web pages, images, videos, and documents.\nAWS S3 buckets can be used to store data in the cloud without exposing it to the public internet. They can also be used to perform data access operations, such as listing, uploading, and downloading files.\n1. AWS S3 Bucket is a cloud-based storage service that allows users to store, manage, and distribute large amounts of data.\n2. AWS S3 Buckets are logical units of storage that are managed by AWS and can be shared by multiple users.\n3. S3'}

In [ ]:
## after the deployment URL will look like this
https://runtime.sagemaker.<region>.amazonaws.com/endpoints/live-finetune-endpoint/invocations